# Customização do Modelo

Esta etapa do trabalho consiste na customização de algum modelo de IA generativa. Neste contexto, será utilizado então o TinyLLama para tentar prever possíveis vulnerabilidades em códigos fonte de contratos inteligentes escritos em solidity. A ideia é que o usuário insira no modelo algum código fonte e ele retorne qual vulnerabilidade esse contrato pode apresentar.

Vale ressaltar que este notebook foi feito utilizando a GPU T4 do Google Colab.

# Installs

In [1]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Usin

In [2]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.0 MB/s eta 0:00:00


In [3]:
!pip install peft transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00


# Imports

In [4]:
import pandas as pd
import torch
import re
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments

In [5]:
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from trl import SFTTrainer

In [6]:
dataset="mwritescode/slither-audited-smart-contracts"
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model="tinyllama-smartcontract-v1"

# Preparação dos dados

A função "prepare_train_data" cria uma nova coluna "text" no dataframe que combina os conteúdos das outras duas colunas em um formato específico, e então converte o DataFrame em um objeto Dataset, que é retornado. Isso é necessário para preparar dados de treinamento para o modelo pois os dados precisam estar em um formato específico.

In [8]:
def prepare_train_data(df):
    data_df = df
    data_df["text"] = data_df[["slither", "source_code"]].apply(lambda x: "<|im_start|>user\n" + x["source_code"] + " <|im_end|>\n<|im_start|>assistant\n" + x["slither"] + "<|im_end|>\n", axis=1)
    data = Dataset.from_pandas(data_df)
    return data

Este dataset contém 5000 contratos, sendo 2500 seguros, ou seja, sem vulnerabilidades e 2500 apresentando pelo menos uma ocorrência da vulnerabilidade descrita como "reentrancy-eth" (associada ao numero 13) nos labels do dataset original.

In [18]:
df = pd.read_parquet('/content/contratos_13.parquet')

In [19]:
df

,source_code,slither
0,pragma solidity ^0.4.23;\n\ncontract PoormansH...,[reentrancy-eth]
1,// SPDX-License-Identifier: GPL-3.0\n\npragma ...,[reentrancy-eth]
2,pragma solidity ^0.4.23;\n\n/*\n!!! THIS CONTR...,"[reentrancy-eth, unchecked-lowlevel]"
3,// SPDX-License-Identifier: MIT\npragma solidi...,"[reentrancy-eth, unused-return]"
4,// SPDX-License-Identifier: MIT\n\npragma soli...,"[uninitialized-state, divide-before-multiply, ..."
...,...,...
4996,/**\n *Submitted for verification at Etherscan...,[safe]
4997,// SPDX-License-Identifier: MIT\n\npragma soli...,[safe]
4998,pragma solidity ^0.4.24;\n/**\n * Marriage\n *...,[safe]
4999,pragma solidity ^0.4.8;\ncontract Token{\n ...,[safe]


In [20]:
#converter lista de palavras para uma unica string
df['slither'] = df['slither'].apply(lambda x: ','.join(x))

df

,source_code,slither
0,pragma solidity ^0.4.23;\n\ncontract PoormansH...,reentrancy-eth
1,// SPDX-License-Identifier: GPL-3.0\n\npragma ...,reentrancy-eth
2,pragma solidity ^0.4.23;\n\n/*\n!!! THIS CONTR...,"reentrancy-eth,unchecked-lowlevel"
3,// SPDX-License-Identifier: MIT\npragma solidi...,"reentrancy-eth,unused-return"
4,// SPDX-License-Identifier: MIT\n\npragma soli...,"uninitialized-state,divide-before-multiply,ree..."
...,...,...
4996,/**\n *Submitted for verification at Etherscan...,safe
4997,// SPDX-License-Identifier: MIT\n\npragma soli...,safe
4998,pragma solidity ^0.4.24;\n/**\n * Marriage\n *...,safe
4999,pragma solidity ^0.4.8;\ncontract Token{\n ...,safe


Diminuindo o tamanho do dataset para apenas 1000 contratos (500 seguros e 500 vulneráveis)

In [21]:
df_safe = df[df['slither'].str.contains('safe')]

# Filtrar linhas que não contêm 'safe' na coluna 'slither'
df_not_safe = df[~df['slither'].str.contains('safe')]

# Amostrar 500 linhas de cada conjunto
df_safe_sample = df_safe.sample(n=500, random_state=1)
df_not_safe_sample = df_not_safe.sample(n=500, random_state=1)

# Concatenar os dois conjuntos de amostras
df = pd.concat([df_safe_sample, df_not_safe_sample]).reset_index(drop=True)

In [22]:
df

,source_code,slither
0,/**\n *Submitted for verification at Etherscan...,safe
1,/**\n *Submitted for verification at Etherscan...,safe
2,pragma solidity ^0.4.11;\n\n\n/**\n * @title O...,safe
3,contract ERC20Basic {\n uint256 public totalS...,safe
4,// File: contracts/ICarbonInventoryControl.sol...,safe
...,...,...
995,/**\n *Submitted for verification at Etherscan...,reentrancy-eth
996,/**\nhttps://t.me/PleasureInu\n\n*/\n\npragma ...,"reentrancy-eth,unused-return"
997,/**\n _\n ...,"reentrancy-eth,unused-return,arbitrary-send"
998,/**\n *Submitted for verification at Etherscan...,"reentrancy-eth,unused-return"


Aplicar a função de remoção de comentários à coluna 'source_code' do DataFrame

In [23]:
def remove_unwanted_characters(code):
    # Remover comentários de linha
    code = re.sub(r'//.*', '', code)
    # Remover comentários de bloco
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    # Remover novas linhas
    code = code.replace('\n', ' ')
    # Remover aspas simples
    code = code.replace("'", '')
    # Remover aspas duplas
    code = code.replace('"', '')
    return code.strip()

df['source_code'] = df['source_code'].apply(remove_unwanted_characters)

In [25]:
data = prepare_train_data(df)

# Modelo

A função "get_model_and_tokenizer" carrega o tokenizer e o modelo de linguagem usando a biblioteca Hugging Face Transformers de acordo com o id passado. Após isso, aplica quantização em 4 bits para reduzir seu tamanho e tentar melhorar a eficiência computacional. Isso foi feito para tentar carregar o modelo de maneira mais eficiente em termos de memória e computação.

In [ ]:
def get_model_and_tokenizer(mode_id):

    tokenizer = AutoTokenizer.from_pretrained(mode_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [ ]:
model, tokenizer = get_model_and_tokenizer(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

# Configurando LoRA

Os trechos de código abaixo configuram e inicializam o ambiente de treinamento utilizando a técnica LoRA. LoRA (Low-Rank Adaptation) é uma técnica projetada para adaptar grandes modelos de aprendizado profundo de maneira eficiente e econômica. Em vez de treinar todos os parâmetros de um modelo grande, LoRA se concentra em ajustar apenas uma pequena parte do modelo. Isso resulta em um treinamento mais rápido e menos dispendioso. O segundo conjunto de argumentos especificamente, ajusta o tamanho do lote e a acumulação de gradiente para otimizar o uso de memória e desempenho. O SFTTrainer é então inicializado com o modelo, conjunto de dados, configuração de LoRA, argumentos de treinamento, tokenizer e outras configurações.

In [ ]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_model,
    per_device_train_batch_size=4,  # reduzir tamanho do batch (estava 16)
    gradient_accumulation_steps=8,  # aumentar para compensar o tamanho do lote reduzido (estava 4)
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    num_train_epochs=3,
    max_steps=250, #steps (estava 500)
    fp16=True,  # precisão mista
    gradient_checkpointing=True  # checkpointing de gradiente
)

In [ ]:
# definindo o trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    peft_config=peft_config,
    dataset_text_field="text",
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=1024
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,0.691800
20,0.587300
30,0.473400
40,0.420000
50,0.407700
60,0.365200
70,0.320200
80,0.362100
90,0.332500
100,0.319500


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_downlo

Step,Training Loss
10,0.691800
20,0.587300
30,0.473400
40,0.420000
50,0.407700
60,0.365200
70,0.320200
80,0.362100
90,0.332500
100,0.319500


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_downlo

TrainOutput(global_step=250, training_loss=0.3573451051712036, metrics={'train_runtime': 5407.0623, 'train_samples_per_second': 1.48, 'train_steps_per_second': 0.046, 'total_flos': 5.069841247862784e+16, 'train_loss': 0.3573451051712036, 'epoch': 8.0})

# Fundindo o LoRA com o modelo básico

In [ ]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)

model_path = "/content/tinyllama-smartcontract-v1/checkpoint-250"

peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

model = peft_model.merge_and_unload()

In [ ]:
model

# Teste

In [ ]:
from transformers import GenerationConfig
from time import perf_counter

In [ ]:
def formatted_prompt(question)-> str:
    return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"

A função generate_response gera uma resposta de um modelo de linguagem com base no input do usuário. Ela formata o input, tokeniza, define as configurações de geração, move os dados para a GPU, executa a geração de texto, decodifica e imprime a resposta gerada, e mede o tempo total de inferência.

In [ ]:
def generate_response(user_input):

  prompt = formatted_prompt(user_input)

  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=12,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()

  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

  outputs = model.generate(**inputs, generation_config=generation_config)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

In [ ]:
dff = pd.read_parquet('/content/contratos_13.parquet')

In [ ]:
xx = dff['source_code'][1]
xx = remove_unwanted_characters(xx)
print(xx)

pragma solidity >=0.7.0 <0.9.0;  contract Storage {           mapping(address => uint) public balances;      function deposit() public payable {         balances[msg.sender] += msg.value;     }      function withdraw() public {         uint256 balance = balances[msg.sender];         balances[msg.sender] = 0;         (bool sent, ) = msg.sender.call{value: balance}();         if (!sent) {             balances[msg.sender] = balance;         }     } }


In [ ]:
generate_response(user_input=xx)

<|im_start|>user
pragma solidity >=0.7.0 <0.9.0;  contract Storage {           mapping(address => uint) public balances;      function deposit() public payable {         balances[msg.sender] += msg.value;     }      function withdraw() public {         uint256 balance = balances[msg.sender];         balances[msg.sender] = 0;         (bool sent, ) = msg.sender.call{value: balance}();         if (!sent) {             balances[msg.sender] = balance;         }     } }<|im_end|>
<|im_start|>assistant: Sure! Here's the updated code with the `with
Time taken for inference: 0.39 seconds


Como é possível observar, o modelo não retorna uma resposta plausível mesmo testando com diversas entradas o que mostra que mais ajustes de hiperparâmetros devem ser feitos.